# MONGO DB PRACTICAL QUESTION 

In [ ]:
#1. Write a Python script to load the Superstore dataset from a CSV file into MongoDB.
#ans:-
import pymongo
import csv

# 1. Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["superstore_db"]
collection = db["orders"]

# 2. Clear old data
collection.delete_many({})

# 3. Read CSV and insert into MongoDB
csv_file_path = 'superstore.csv'
data_to_insert = []

# Open file with the correct encoding
with open(csv_file_path, mode='r', encoding='latin-1') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        # Convert numeric fields
        try:
            row['Sales'] = float(row.get('Sales', 0))
            row['Quantity'] = int(row.get('Quantity', 0))
            row['Discount'] = float(row.get('Discount', 0))
            row['Profit'] = float(row.get('Profit', 0))
        except (ValueError, TypeError):
            pass 
        data_to_insert.append(row)

if data_to_insert:
    collection.insert_many(data_to_insert)
    print(f"Successfully loaded {len(data_to_insert)} documents.")




In [ ]:
#2. Retrieve and print all documents from the Orders collection.
#ans:-
# find({}) with an empty dictionary retrieves all documents
all_orders = collection.find({})

# The result is a cursor, which we can iterate over
for order in all_orders:
    print(order)

In [ ]:
#3. Count and display the total number of documents in the Orders collection
#ans:-
# count_documents({}) with an empty filter counts all documents
total_documents = collection.count_documents({})

print(f"Total number of documents in the Orders collection: {total_documents}")

In [ ]:
#4. Write a query to fetch all orders from the "West" region. 
#ans:-
# The filter {"Region": "West"} specifies the condition
west_region_orders = collection.find({"Region": "West"})

print("Orders from the West region:")
for order in west_region_orders:
    print(order)

In [ ]:
#5. Write a query to find orders where Sales is greater than 500.
#ans:-
# The $gt operator stands for "greater than"
high_sales_orders = collection.find({"Sales": {"$gt": 500}})

print("Orders with Sales greater than 500:")
for order in high_sales_orders:
    print(order)

In [ ]:
#6. Fetch the top 3 orders with the highest Profit. 
#ans:-
# We use sort() to order the results and limit() to get the top N
# pymongo.DESCENDING or -1 is used for descending order
top_3_profit_orders = collection.find({}).sort("Profit", pymongo.DESCENDING).limit(3)

print("Top 3 orders with the highest profit:")
for order in top_3_profit_orders:
    print(order)

In [ ]:
#7. Update all orders with Ship Mode as "First Class" to "Premium Class."
#ans:-
# The first dictionary is the filter, the second is the update operation
# $set is used to update the value of a field
update_result = collection.update_many(
    {"Ship Mode": "First Class"},
    {"$set": {"Ship Mode": "Premium Class"}}
)

print(f"Matched {update_result.matched_count} documents and modified {update_result.modified_count} documents.")


In [ ]:
#8. Delete all orders where Sales is less than 50
#ans:-
# The $lt operator stands for "less than"
delete_result = collection.delete_many({"Sales": {"$lt": 50}})

print(f"Deleted {delete_result.deleted_count} documents where Sales was less than 50.")

In [ ]:
#9. Use aggregation to group orders by Region and calculate total sales per region.
#ans:-
# The aggregation pipeline is a list of stages
pipeline = [
    {
        "$group": {
            "_id": "$Region",  # Group by the "Region" field
            "TotalSales": {"$sum": "$Sales"}  # Calculate the sum of "Sales" for each group
        }
    },
    {
        "$sort": {"TotalSales": -1} # Optional: sort by total sales
    }
]

sales_by_region = collection.aggregate(pipeline)

print("Total sales per region:")
for region_data in sales_by_region:
    print(region_data)

In [ ]:
#10. Fetch all distinct values for Ship Mode from the collection
#ans:-
# The distinct() method returns a list of unique values for a given field
distinct_ship_modes = collection.distinct("Ship Mode")

print("Distinct Ship Modes:")
print(distinct_ship_modes)

In [ ]:
#11. Count the number of orders for each category.
#ans:-
# This aggregation pipeline groups by "Category" and counts the documents in each group
pipeline = [
    {
        "$group": {
            "_id": "$Category",
            "OrderCount": {"$sum": 1} # For each document in the group, add 1 to the count
        }
    }
]

orders_per_category = collection.aggregate(pipeline)

print("Number of orders per category:")
for category_count in orders_per_category:
    print(category_count)

In [ ]:
#  Close the connection
client.close()
print("Connection closed.")